In [1]:
import pandas as pd
import numpy as np

In [2]:
barometric = pd.read_csv("barometric.csv", names=["row","unix_time", "date", "time", "barometric", "unnamned"])
direction = pd.read_csv("direction.csv", names=["row", "unix_time", "date", "time", "direction", "unnamed"])
humidity = pd.read_csv("humidity.csv", names=["row", "unix_time", "date", "time", "humidity", "unnamed"])
radiation = pd.read_csv("radiation.csv", names=["row", "unix_time", "date", "time", "radiation", "unnamed"])
speed = pd.read_csv("speed.csv", names=["row", "unix_time", "date", "time", "speed", "unnamed"])
# sunrise = pd.read_csv("sunrise.csv", names=["row", "unix_time", "date", "time", "sunrise", "unnamed"])
# sunset = pd.read_csv("sunset.csv", names=["row", "unix_time", "date", "time", "sunset", "unnamed"])
temperature = pd.read_csv("temperature.csv", names=["row", "unix_time", "date", "time", "temp", "unnamed"])

In [3]:
print barometric.size
print direction.size
print humidity.size
print radiation.size
print speed.size
# print sunrise.size
# print sunset.shape
print temperature.shape

198798
198798
198798
198798
198798
(33133, 6)


In [ ]:
barometric.head()

In [4]:
features = [barometric, direction, humidity, radiation, speed, temperature]
for feature in features:
    feature.drop(feature.columns[[0,2,3,5]], axis=1, inplace=True)
# feature = reduce(lambda left, right: pd.merge(left, right,on="unix_time"), features)
# features = iter(features)
# feature = next(features)
# for df_ in features:
#     feature = feature.merge(df_, on='unix_time')
# fea = pd.merge(barometric,sunrise, on="unix_time")
# fea.count(axis=0)
# feature.count(axis=0)

In [ ]:
fea.head()

In [5]:
fea = features[0]
for feat in features[1:]:
    fea= pd.merge(fea,feat, on="unix_time", how="outer")
fea.count(axis=0)    

unix_time     33133
barometric    33133
direction     33133
humidity      33133
radiation     33133
speed         33133
temp          33133
dtype: int64

In [6]:
fea.head()

,unix_time,barometric,direction,humidity,radiation,speed,temp
0,1475315718,30.46,151.95,80,1.27,7.87,48
1,1475315423,30.46,176.14,79,1.25,7.87,48
2,1475315124,30.46,177.54,79,1.25,9.00,48
3,1475314821,30.46,167.56,79,1.25,13.50,48
4,1475314522,30.47,177.81,79,1.27,15.75,48


In [7]:
fea.shape

(33133, 7)

In [10]:
Y = fea["radiation"]
Y.head()
X =fea[["barometric", "humidity", "speed", "temp"]]
X.head()

,barometric,humidity,speed,temp
0,30.46,80,7.87,48
1,30.46,79,7.87,48
2,30.46,79,9.00,48
3,30.46,79,13.50,48
4,30.47,79,15.75,48


In [11]:
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()
X = min_max_scaler.fit_transform(X)

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [20]:
# import xgboost as xgb
# xgbmat = xgb.DMatrix(X_train,y_train)
# params = {"objective": "reg:linear", "booster":"gblinear"}
# gbm = xgb.train(dtrain=xgbmat,params=params)
# testdmat = xgb.DMatrix(X_test)
# # from sklearn.metrics import accuracy_score
# y_pred = gbm.predict(testdmat)
# accuracy = gbm.score(y_pred, y_test)
# print accuracy

In [ ]:
from sklearn.linear_model import LinearRegression
clf = LinearRegression()
clf.fit(X_train, y_train)
print clf.score(clf.predict(X_test), y_test)

AttributeError: 'Booster' object has no attribute 'score'